# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 

In [7]:
import numpy as np
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import json
from nltk import Tree
from spacy import displacy
import spacy
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from featurizers import GrammarVectorizer, make_document
from contextlib import contextmanager
warnings.filterwarnings("ignore", category=FutureWarning)

In [11]:
@contextmanager
def temp_change_dir(path:str):
    """Changes to a new directory specified by given path, and then reverts back to the old directory"""
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
        
def load_pan22() -> pd.DataFrame:
    """Loads PAN22 data as a dataframe"""
    with temp_change_dir("../../"):
        return pd.read_json("data/pan22/preprocessed/pan22_preprocessed.jsonl", lines=True)

In [19]:
pan = load_pan22()
pan_docs = pan["fullText"]

g2v = GrammarVectorizer()
DVs = g2v.vectorize_episode(pan_docs.to_list(), return_obj=True)

In [26]:
dv1= DVs[0]

def get_tabular_dataset(dvs) -> pd.DataFrame:
    """Applies feature vectors to a list of documents and stores everything in a DataFrame"""
    
    
    featurizer_names = g2v.get_config()

    for dv in dvs:
        
    
    

    
get_tabular_dataset(DVs)
    

['pos_unigrams', 'pos_bigrams', 'func_words', 'punc', 'letters', 'common_emojis', 'document_stats', 'dep_labels', 'mixed_bigrams', 'morph_tags']
